In [1]:
########## extract dataset for validity tests; remove cue words where guess words between humans and machines overlap
import numpy as np
import json

cue_words_list = np.load('Data_all/all_cue_words_october_1.npy')
CueGuessWords = {}

for w in cue_words_list:
    #print(w)
    CueGuessWords[w] = {'human':[],'AI':[]}

    
numfiles = 150
numfolders = 5

for identity in ['human','AI']:
    for fo in range(numfolders):
        for fi in range(numfiles):
            filename = identity + '/' + str(fo) + '/' + str(fi) + '.html'

            with open('./dataset/' + filename) as f:
                lines = f.readlines()
                #print(lines)
                cueword = lines[0].split('<p>Cue: ')[1].split(';')[0]
                #print(cueword)
                guessword = lines[0].split('; Association: ')[1].split(' </p>')[0]
                #print(guessword)
                CueGuessWords[cueword][identity].append(guessword)

#print(CueGuessWords)

for w in cue_words_list:
    #print(w)
    strlistH = CueGuessWords[w]['human']
    strlistM = CueGuessWords[w]['AI']
    flag = bool(set(strlistH).intersection(strlistM))
#     if not flag:
#         print(w)
#         print(strlistH)
#         print(strlistM)
    CueGuessWords[w]['flag'] = flag

#flag: 1 -> invalid; NOT use
#flag: 0 -> valid; use
with open("./savedResults/DataValidity.json", "w") as fp:
    json.dump(CueGuessWords,fp) 

In [ ]:
import csv
import json
import sqlite3
import numpy as np
import pandas as pd


"""
link response to model number
"""
def find_real_from_response(response):
    response = response.split('.')[0]; response = response.split('_'); 
    model = response[-2]
    #if model == 'AI' :
    #try :
    #int(response[-2])
    model_num =  int(response[-1])
    #except:
    #else:
    #model_num = 0; model = response[-2]
    return model, model_num

"""
numbers to models names
"""
#dict_numbers_to_model_name = np.load('Data_all/dict_numbers_to_model_name.npy',allow_pickle=True).item() # the location is C:\Users\noga mudrik\Documents\turing_associations\all_important_code_files
# dict_numbers_to_model_name = {0: 'word2vec',
#  1: 'gpt2',
#  2: 'gpt3 (embedding)',
#  3: 'gpt3 (prompt curie)',
#  4: 'gpt3 (prompt davinvci)'}
## Elisa: Ok, so the convention is the following: a = human, b = word2vec, c = gpt2, d = gpt3 (embedding), e = gpt3 (prompt curie), f = gpt3 (prompt davinci), g = catch trials
dict_numbers_to_model_name = {'b': 'word2vec',
 'c': 'gpt2',
 'd': 'gpt3 (embedding)',
 'e': 'gpt3 (prompt curie)',
 'f': 'gpt3 (prompt davinvci)', 
 'g': 'other'}

possible_models = dict_numbers_to_model_name.values()
print(dict_numbers_to_model_name)

dict_gender_results = np.load('Data_all/dict_gender_results.npy', allow_pickle=True).item()

cue_words_list = np.load('Data_all/all_cue_words_october_1.npy')

results = {'inlab':[], 'inlab_gender':[], 'inlab_nogender':[], 'mturk':[], 'machine':[], 'combined':[]}
results = {'AIexperts': []}


############# for jason file processing (inlab)
jsonpath = './Data_all/'
# filenames = {'inlab': ['word_association_inlab_gender_15','word_association_inlab_no_gender_15'],
#              'inlab_gender': ['word_association_inlab_gender_15'],
#              'inlab_nogender': ['word_association_inlab_no_gender_15']}
filenames = {'AIexperts': ['participants']}

for key in filenames:
    print("=======================")
    print(key)

    count = 0
    trials_dict = {}
    workercount = 0
    
    for dbfilename in filenames[key]:
        #print("=======================")
        #print(jasonfilename)
        conn = sqlite3.connect(jsonpath + dbfilename + ".db")
        cursor = conn.cursor()
        cursor.execute("select * from TuringAssociationAI_turing;")        
        all_data = cursor.fetchall()
                
        for i in range(0, len(all_data)):
            if (all_data[i][15] == 3 and all_data[i][17] is not None): #check status 
                data_dict = json.loads(all_data[i][17])
                workercount += 1
                
                for dat in data_dict['data']:
                    if dat['trialdata']['phase'] == 'TEST':
                        count += 1
                        trial_data = {}
                        trial_data['workerID'] = data_dict['workerId']
                        #print(data_dict['workerId'])
                        trial_data['workerData'] = data_dict['questiondata']
                        curres = dat['trialdata']
                        rt = curres['rt'] ###
                        if 'hit' in curres.keys():
                            hit = curres['hit'] ###
                        elif 'page_name' in curres.keys():
                            page_name = curres['page_name']
                            hit = page_name.split('/')[-1]
                        trial = curres['trial']
                        
                        guess = curres['Aclass']; #print(guess); print(model_real)
                        response_speaker = guess ###
                        if response_speaker == 'AI':
                            response_speaker = 'machine'
                        
                        #print(curres['prompt'])
                        groundtruth, model_num = find_real_from_response(hit)#response
                        
                        if  groundtruth == 'a':
                            groundtruth = 'human'
                            #curres_split = curres['AssociationID'][:-5].split('/')
                            #real_gender = dict_gender_results[int(curres_split[-2])][int(curres_split[-1])]
                            machine_groundtruth = ''
                            human_groundtruth = 'human'
                        else:
                            #real_gender = ''
                            machine_groundtruth = dict_numbers_to_model_name[groundtruth]
                            human_groundtruth = ''
                            groundtruth = 'machine'
                            #print(machine_groundtruth)
                        
                        
                        imageID = int(hit.split('.')[0].split('_')[-3])
                        caption = cue_words_list[imageID] # cue word
                        
                        if CueGuessWords[caption]['flag']:
                            count = count - 1
                            #print('invalid')
                            continue
                        
                        counterbalance = 0

                        phase = curres['phase']
                        
                        if 'guess_gender' in curres.keys():
                            guess_gender = curres['guess_gender']
                            if guess_gender == 'm': 
                                guess_gender_long = 'male' # guess gender
                            else:
                                guess_gender_long = 'female' # guess gender

                            trialData_save = {'rt':rt, 'hit':hit, 'trial':trial, 
                                          'response_speaker':response_speaker,
                                          'machine_groundtruth':machine_groundtruth,'imageID':imageID,
                                          'caption': caption, 'counterbalance':counterbalance,
                                          'groundtruth':groundtruth,
                                          'phase':phase, 'human_groundtruth':human_groundtruth}
                                          #'real_gender':real_gender, 'response_gender':guess_gender_long
                        else:
                            trialData_save = {'rt':rt, 'hit':hit, 'trial':trial, 
                                          'response_speaker':response_speaker,
                                          'machine_groundtruth':machine_groundtruth,'imageID':imageID,
                                          'caption': caption, 'counterbalance':counterbalance,
                                          'groundtruth':groundtruth,
                                          'phase':phase, 'human_groundtruth':human_groundtruth}
                                          #'real_gender':real_gender}                       
                        
                        
                        trial_data['trialData'] = trialData_save
                        trials_dict['Trial_'+str(count)] = trial_data

        conn.close()
    results[key] = trials_dict
    #print(trials_dict)
        #json.dump(trials_dict, open("./turing_inlab_no_gender_img_caption_half.json","w"))
        
    print(workercount, ' workers')


{'b': 'word2vec', 'c': 'gpt2', 'd': 'gpt3 (embedding)', 'e': 'gpt3 (prompt curie)', 'f': 'gpt3 (prompt davinvci)', 'g': 'other'}
AIexperts


NameError: name 'CueGuessWords' is not defined

In [4]:
print('All AI expert data is extracted.')
for key in results:
    print('There are [' + key + '; ' + str(len(results[key])) + ']')
    
#save the compiled results as jason
with open("./savedResults_elisa/compiled_AIexperts.json", "w") as fp:
    json.dump(results,fp) 
    
print('Compilation completed! Results are saved to folder!')

All AI expert data is extracted.
There are [AIexperts; 391]
Compilation completed! Results are saved to folder!


In [9]:
########### mturk jason files
dbfile = 'Data_all/participants_28_mturk.db'
cnx = sqlite3.connect(dbfile)
df = pd.read_sql_query("SELECT * FROM 'TuringAssociationAI_turing'", cnx)#TuringAssociationAI_turing

## pre-process db files; NOT all db entries are usable...
vals = df['beginhit'].values
locs_vals = [i for i,v in enumerate(vals) if (v.startswith('2022-10-27') or v.startswith('2022-10-28'))]
df = df.iloc[locs_vals,:]
df = df.sort_values(by='beginhit')#['beginhit']
df = df.iloc[-80:,:]

res_per_subject = {i: json.loads(df['datastring'][i])  
                        for i in df['datastring'].index.values 
                        if (df['datastring'][i])} 

key = 'mturk'
count = 0
trials_dict = {}
workercount = 0

for sub,res_per_subject_subject in res_per_subject.items():
    workerData = res_per_subject_subject['questiondata']
    workerID = res_per_subject_subject['workerId']
    flag = 0
    for res_per_subject_subject_spec in res_per_subject_subject['data']:
        if 'trialdata' in res_per_subject_subject_spec:           
            if 'prompt' in res_per_subject_subject_spec['trialdata']:
                flag = 1
                curres = res_per_subject_subject_spec['trialdata']
                
                count += 1
                trial_data = {}
                trial_data['workerID'] = workerID
                #print(data_dict['workerId'])
                trial_data['workerData'] = workerData
                
                rt = curres['rt'] ###
                hit = curres['hit'] ###
                trial = curres['trial']

                guess = curres['guess_decision']; #print(guess); print(model_real)
                response_speaker = guess ###
                if response_speaker == 'AI':
                    response_speaker = 'machine'

                #print(curres['prompt'])
                groundtruth, model_num = find_real_from_response(curres['prompt'])#response

                if  groundtruth == 'human':

                    curres_split = curres['AssociationID'][:-5].split('/')
                    real_gender = dict_gender_results[int(curres_split[-2])][int(curres_split[-1])]
                    machine_groundtruth = ''
                    human_groundtruth = 'human'
                else:
                    groundtruth = 'machine'
                    real_gender = ''
                    machine_groundtruth = dict_numbers_to_model_name[model_num]
                    human_groundtruth = ''
                    #print(machine_groundtruth)


                imageID = int(curres['prompt'].split('.')[0].split('/')[-1])
                caption = cue_words_list[imageID] # cue word
                counterbalance = 0

                if CueGuessWords[caption]['flag']:
                    count = count - 1
                    continue
                
                phase = curres['phase']

                if 'guess_gender' in curres.keys():
                    guess_gender = curres['guess_gender']
                    if guess_gender == 'm': 
                        guess_gender_long = 'male' # guess gender
                    else:
                        guess_gender_long = 'female' # guess gender

                    trialData_save = {'rt':rt, 'hit':hit, 'trial':trial, 
                                  'response_speaker':response_speaker,
                                  'machine_groundtruth':machine_groundtruth,'imageID':imageID,
                                  'caption': caption, 'counterbalance':counterbalance,
                                  'groundtruth':groundtruth,
                                  'phase':phase, 'human_groundtruth':human_groundtruth ,
                                  'real_gender':real_gender, 'response_gender':guess_gender_long}
                else:
                    trialData_save = {'rt':rt, 'hit':hit, 'trial':trial, 
                                  'response_speaker':response_speaker,
                                  'machine_groundtruth':machine_groundtruth,'imageID':imageID,
                                  'caption': caption, 'counterbalance':counterbalance,
                                  'groundtruth':groundtruth,
                                  'phase':phase, 'human_groundtruth':human_groundtruth,
                                  'real_gender':real_gender}                       


                trial_data['trialData'] = trialData_save
                trials_dict['Trial_'+str(count)] = trial_data
        
    if flag == 1:
        workercount = workercount + 1
        
results[key] = trials_dict  
#print(trials_dict)
print(key)
print(workercount, ' workers')

mturk
71  workers
mturk
71  workers


In [10]:
########### AI jason files
#### pending Noga's results; remove invalid cue words

key = 'machine'
count = 0
trials_dict = {}

with open("./Data_all/dict_cue_guess_full_who_Said_1500.json", 'r') as f:
    cuelist = json.load(f)

json_file_path_list = ['AI_judge_new_structure_for_dict_WORD2VEC_1500_judge',
                  'AI_judge_new_structure_for_dict_GPT2_1500_judge',
                  'AI_judge_new_structure_for_dict_GPT3_1500_judge']

#json_file_path_list = ['AI_judge_new_structure_for_dict_WORD2VEC_1500_judge']
    
aijudgename = ['word2vec','gpt2','gpt3']
taboo_machine_groundtruth = ['word2vec','gpt2','gpt3 (embedding)']
#the taboo phrase exlucdes trials where ai judge is the same as the ai agent

for i, json_file_path in enumerate(json_file_path_list): 
    judgename = aijudgename[i]
    tabooname = taboo_machine_groundtruth[i]
    
    with open(jsonpath + json_file_path + '.json', 'r') as j:
        contents = json.loads(j.read())
        #results['machine'] = contents
        #print(contents)
        for trialid in contents:
            
            count += 1
            trial_data = {}
            trial_data['workerID'] = judgename
            
            cueword = cuelist[trialid]['cue']
            if CueGuessWords[cueword]['flag']:
                count = count - 1
                #print('happend!')
                continue
                    
            trialinfor = contents[trialid]
            trial = trialinfor['trial']
            response_speaker = trialinfor['response_speaker']
            machine_groundtruth = trialinfor['machine_groundtruth']
            
            if response_speaker == 'AI':
                response_speaker = 'machine'
            else:
                response_speaker = 'human'
            
            if machine_groundtruth == 'human':
                groundtruth = 'human'
                machine_groundtruth = ''
                human_groundtruth = 'human'
            else:
                groundtruth = 'machine'
                human_groundtruth = ''
                if machine_groundtruth == ' gpt3 (embedding)':
                    machine_groundtruth = 'gpt3 (embedding)'
            
            if tabooname == machine_groundtruth:
                count = count - 1
                #print('happend; taboo')
                continue
                
            trialData_save = {'trial':trial,
                                  'caption': cueword,
                                  'response_speaker':response_speaker,
                                  'machine_groundtruth':machine_groundtruth,                                  
                                  'groundtruth':groundtruth,
                                  'human_groundtruth':human_groundtruth}
            #print(trialData_save)
            trial_data['trialData'] = trialData_save
            trials_dict['Trial_'+str(count)] = trial_data
        
    
results['machine'] = trials_dict


In [11]:
#save combined results
count = 0
trials_dict = {}
for ke in ['inlab','mturk']:
    for it in results[ke]:
        count = count + 1
        trialin = results[ke][it]
        trials_dict['Trial_'+str(count)] = trialin
        
results['combined'] = trials_dict

In [12]:
print('All data is extracted.')
for key in results:
    print('There are [' + key + '; ' + str(len(results[key])) + ']')
    
#save the compiled results as jason
with open("./savedResults/compiled.json", "w") as fp:
    json.dump(results,fp) 
    
print('Compilation completed! Results are saved to folder!')

All data is extracted.
There are [inlab; 843]
There are [inlab_gender; 415]
There are [inlab_nogender; 428]
There are [mturk; 1930]
There are [machine; 2241]
There are [combined; 2773]
All data is extracted.
There are [inlab; 843]
There are [inlab_gender; 415]
There are [inlab_nogender; 428]
There are [mturk; 1930]
There are [machine; 2241]
There are [combined; 2773]
Compilation completed! Results are saved to folder!
Compilation completed! Results are saved to folder!
